In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [2]:
df = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
#filling NULL values with empty string
df=df.fillna('')
test=test.fillna('')

In [6]:
# We will be only using title and author name for prediction
# Creating new coolumn total concatenating title and author
df['total'] = df['title']+' '+df['author']
test['total']=test['title']+' '+test['author']

In [8]:
X = df.drop('label',axis=1)
y=df['label']
print(X.shape)
print(y.shape)

(20800, 5)
(20800,)


In [26]:
print(X.head())
print(y.head())

   id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  \
0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1  Ever get the feeling your life circles the rou...   
2  Why the Truth Might Get You Fired October 29, ...   
3  Videos 15 Civilians Killed In Single US Airstr...   
4  Print \nAn Iranian woman has been sentenced to...   

                                               total  
0  House Dem Aide: We Didn’t Even See Comey’s Let...  
1  FLYNN: Hillary Clinton, Big Woman on Campus - ...  
2  Why the Truth 

In [9]:
#Choosing vocabulary size to be 5000 and copying data to msg for further cleaning
voc_size = 5000
msg = X.copy()
msg_test = test.copy()

In [29]:
msg

,id,title,author,text,total
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Why the Truth Might Get You Fired Consortiumne...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Iranian woman jailed for fictional unpublished...
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Rapper T.I.: Trump a ’Poster Child For White S...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Macy’s Is Said to Receive Takeover Approach by...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","NATO, Russia To Hold Parallel Exercises In Bal..."


In [10]:
#Downloading stopwords 
#Stopwords are the words in any language which does not add much meaning to a sentence.
#They can safely be ignored without sacrificing the meaning of the sentence.
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emmanuel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
#We will be using Stemming here
#Stemming map words to their root forms
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [42]:
d = re.sub('[^a-zA-Z]',' ',msg['total'][0])
d = d.lower()
print(d)
d = d.split()
print(d)
d = [ps.stem(word) for word in d if not word in stopwords.words('english')]
print(d)
d = ' '.join(d)
print(d)

house dem aide  we didn t even see comey s letter until jason chaffetz tweeted it darrell lucus
['house', 'dem', 'aide', 'we', 'didn', 't', 'even', 'see', 'comey', 's', 'letter', 'until', 'jason', 'chaffetz', 'tweeted', 'it', 'darrell', 'lucus']
['hous', 'dem', 'aid', 'even', 'see', 'comey', 'letter', 'jason', 'chaffetz', 'tweet', 'darrel', 'lucu']
hous dem aid even see comey letter jason chaffetz tweet darrel lucu


In [47]:
e = [one_hot(words,voc_size)for words in d]
print(e)

[[2603], [3541], [3816], [4071], [], [2722], [662], [1564], [], [4816], [4655], [2722], [], [662], [3622], [662], [1480], [], [4071], [662], [662], [], [1434], [3541], [1564], [662], [3888], [], [3602], [662], [211], [211], [662], [184], [], [2842], [4816], [4071], [3541], [1480], [], [1434], [2603], [4816], [370], [370], [662], [211], [4115], [], [211], [2708], [662], [662], [211], [], [2722], [4816], [184], [184], [662], [3602], [], [3602], [3816], [1434], [3816]]


In [51]:
f = pad_sequences(e,padding='pre',maxlen=25)

[[   0    0    0 ...    0    0 2603]
 [   0    0    0 ...    0    0 3541]
 [   0    0    0 ...    0    0 3816]
 ...
 [   0    0    0 ...    0    0 3816]
 [   0    0    0 ...    0    0 1434]
 [   0    0    0 ...    0    0 3816]]


In [12]:
# [^a-zA-Z] is a regular expression that matches any character NOT part of the characters 
# a-z or A-Z

#Applying stemming and some preprocessing
for i in range(len(msg)):
  review = re.sub('[^a-zA-Z]',' ',msg['total'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [13]:
#Applying stemming and some preprocessing for test data
corpus_test = []
for i in range(len(msg_test)):
  review = re.sub('[^a-zA-Z]',' ',msg_test['total'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus_test.append(review)

In [43]:
# Converting to one hot representation
onehot_rep = [one_hot(words,voc_size)for words in corpus]
onehot_rep_test = [one_hot(words,voc_size)for words in corpus_test]

In [15]:
#Padding Sentences to make them of same size
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=25)
embedded_docs_test = pad_sequences(onehot_rep_test,padding='pre',maxlen=25)

In [ ]:
#Train a model on y ~ embedded_docs

In [75]:
y.shape

(20800,)

In [70]:
embedded_docs.shape

(20800, 25)

In [77]:
embedded_docs

array([[   0,    0,    0, ..., 4603, 1262, 3635],
       [   0,    0,    0, ...,  220, 2842,  624],
       [   0,    0,    0, ..., 1512, 1674,  205],
       ...,
       [   0,    0,    0, ..., 1245, 3366, 1510],
       [   0,    0,    0, ...,   67, 3161,  208],
       [   0,    0,    0, ...,  338, 2385, 1506]], dtype=int32)

In [87]:
pd.DataFrame(embedded_docs).to_csv("~/Desktop/Kaggle/FakeNews/Data/embedded.csv")
pd.DataFrame(embedded_docs_test).to_csv("~/Desktop/Kaggle/FakeNews/Data/embedded_test.csv")

In [ ]:
# use that model to predicg embedded_docs_test